In [7]:
import pymorphy2
import os
import re
import string

from bs4 import BeautifulSoup
from nltk import ngrams
from nltk.corpus import stopwords

In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
# p = morph.parse('викинги')[0]
# p.normal_form

In [10]:
raw_sites_directory = 'sites'
parsed_sites_directory = 'parsed_sites_bigrams'

punctuation = string.punctuation+"«"+"»"+"”"+"“"+"\ufeff"
translation_table = str.maketrans({key: ' ' for key in punctuation})
russian_stopwords = stopwords.words('russian')
n = 2

def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))


# def tokenize(raw):
#     return [
#         word
#         for word in raw.strip().lower().translate(translation_table).split(' ')
#         if raw
#         and isinstance(raw, str)
#         and word not in russian_stopwords
#         and len(word) > 1
#         and has_cyrillic(word)
#     ]

def tokenize(raw):
    return ' '.join([
        word
        for word in raw.strip().lower().translate(translation_table).split(' ')
        if raw
        and isinstance(raw, str)
        and word not in russian_stopwords
        and len(word) > 1
        and has_cyrillic(word)
    ])

def normalize(word):
    return morph.parse(word.lower().strip())[0].normal_form 


def cleanme(html):
    soup = BeautifulSoup(html)
    return soup.text

In [11]:
def site_to_bow(fname):
    if not fname.startswith('http'):
        return
    path = '{}/{}'.format(raw_sites_directory, fname)
    fout = '{}/{}'.format(parsed_sites_directory, fname)
    with open(path) as fp:
        try:
            fdata = fp.read()
        except UnicodeDecodeError:
            print(fname)
            raise
    clean = ngrams(tokenize(cleanme(fdata)).split(), n)
    words = [str(sorted((normalize(a), normalize(b)),)) for a, b in clean]
    with open(fout, 'w') as fw:
        fw.write('\n'.join(words))

In [12]:
if not os.path.exists(parsed_sites_directory):
    os.makedirs(parsed_sites_directory)

ready = {ff_name for ff_name in os.listdir(parsed_sites_directory)}

for fname in os.listdir(raw_sites_directory):
    if fname in ready:
        continue
    site_to_bow(fname)

/Users/chikachoff/virtualenvs/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: generator 'ngrams' raised StopIteration
  del sys.path[0]
